# function_calling案例实战

## 1. 天气预报案例实战

### 1.1 准备环境

In [1]:
! pip install scipy tenacity tiktoken termcolor openai requests


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

#client.models.list()

### 1.2 封装模型调用

In [71]:
# 定义一个函数chat_completion_request，主要用于发送 聊天补全 请求到OpenAI服务器
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e



### 1.3 定义打印函数

In [72]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    # 遍历消息列表
    for message in messages:
            # 如果消息的角色是"system"，则用红色打印“content”
            if isinstance(message, dict) and message["role"] == "system":
                print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
            # 如果消息的角色是"user"，则用绿色打印“content”
            elif isinstance(message, dict) and message["role"] == "user":
                print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
            # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
            elif hasattr(message, 'role') and message.role == "assistant" and message.function_call:
                print(colored(f"assistant: {message.function_call}\n", role_to_color[message.role]))
            # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
            elif hasattr(message, 'role') and message.role == "assistant" and not message.function_call:
                print(colored(f"assistant: {message.content}\n", role_to_color[message.role]))
            elif isinstance(message, dict) and message["role"] == "function":
            # 如果消息的角色是"function"，则用品红色打印“function”
                print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### 1.4 定义function函数

In [73]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的函数
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",# 函数的名称
            "description": "Get the current weather",# 函数的描述
            "parameters": {# 定义该函数需要的参数
                "type": "object",
                "properties": {# 参数的属性
                    "location": {# 地点参数
                        "type": "string",# 参数类型为字符串
                        "description": "The city and state, e.g. San Francisco, CA",# 参数的描述
                    },
                    "format": {# 温度单位参数
                        "type": "string",# 参数类型为字符串
                        "enum": ["celsius", "fahrenheit"],# 摄氏度，华氏 参数的取值
                        "description": "The temperature unit to use. Infer this from the users location.",# 参数的描述
                    },
                },
                "required": ["location", "format"],# 该函数需要的必要参数
            },
        }
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的函数
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

### 1.5 函数调用：咨询今天天气

In [79]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
#assistant_message
# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure, could you please provide the location for which you would like to know the weather?



### 1.6 函数调用：回复当前地点

In [80]:
messages.append({"role": "user", "content": "I'm in ShenZhen, China."})
#大模型自己选择是否使用函数
#chat_response = chat_completion_request(messages, tools=tools)
#强制使用函数
chat_response = chat_completion_request(messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_current_weather"}})
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
#assistant_message
# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure, could you please provide the location for which you would like to know the weather?

user: I'm in ShenZhen, China.

assistant: None



### 设置问答样例，用于咨询未来5天的天气

In [43]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
#assistant_message
# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Glasgow, Scotland over the next x days

assistant: Sure, I can help you with that. Could you please specify the value of `x`?



In [ ]:
### 咨询未来5天天气

In [44]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ihsCi6R5mIRHQPhXPQC9DLex', function=Function(arguments='{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}', name='get_n_day_weather_forecast'), type='function')]))

In [ ]:
### 强制使用函数

In [45]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5ll1eH1bGp2QvNziUUe5wnzD', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}', name='get_n_day_weather_forecast'), type='function')])

In [ ]:
### 强制不使用函数

In [46]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.choices[0].message

ChatCompletionMessage(content='{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}', role='assistant', function_call=None, tool_calls=None)

## 2. 基于大模型实现数据平台查询

### 1. 定义数据库信息

In [56]:
import sqlite3

conn = sqlite3.connect("data/chinook.db")
print("Opened database successfully")

Opened database successfully


首先定义三个函数get_table_names、get_column_names和get_database_info，用于从数据库连接对象中获取数据库的表名、表的列名以及整体数据库的信息。

In [57]:
def get_table_names(conn):
    """返回一个包含所有表名的列表"""
    table_names = []  # 创建一个空的表名列表
    # 执行SQL查询，获取数据库中所有表的名字
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # 遍历查询结果，并将每个表名添加到列表中
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names  # 返回表名列表


def get_column_names(conn, table_name):
    """返回一个给定表的所有列名的列表"""
    column_names = []  # 创建一个空的列名列表
    # 执行SQL查询，获取表的所有列的信息
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    # 遍历查询结果，并将每个列名添加到列表中
    for col in columns:
        column_names.append(col[1])
    return column_names  # 返回列名列表


def get_database_info(conn):
    """返回一个字典列表，每个字典包含一个表的名字和列信息"""
    table_dicts = []  # 创建一个空的字典列表
    # 遍历数据库中的所有表
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)  # 获取当前表的所有列名
        # 将表名和列名信息作为一个字典添加到列表中
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  # 返回字典列表

In [58]:
# 获取数据库信息，并存储为字典列表
database_schema_dict = get_database_info(conn)

# 将数据库信息转换为字符串格式，方便后续使用
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)
database_schema_string

'Table: albums\nColumns: AlbumId, Title, ArtistId\nTable: sqlite_sequence\nColumns: name, seq\nTable: artists\nColumns: ArtistId, Name\nTable: customers\nColumns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId\nTable: employees\nColumns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email\nTable: genres\nColumns: GenreId, Name\nTable: invoices\nColumns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total\nTable: invoice_items\nColumns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity\nTable: media_types\nColumns: MediaTypeId, Name\nTable: playlists\nColumns: PlaylistId, Name\nTable: playlist_track\nColumns: PlaylistId, TrackId\nTable: tracks\nColumns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice\nTable: sqlite_stat1\nColumn

### 2.定义 functions让 GPT 模型帮我们构造一个完整的 SQL 查询

In [60]:
# 定义一个功能列表，其中包含一个功能字典，该字典定义了一个名为"ask_database_get_sql"的功能，用于回答用户关于音乐的问题
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database_get_sql",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

### 3. 执行SQL语句（根据大模型返回的SQL，执行查询操作）

In [61]:
def ask_database(conn, query):
    """使用 query 来查询 SQLite 数据库的函数。"""
    try:
        results = str(conn.execute(query).fetchall())  # 执行查询，并将结果转换为字符串
    except Exception as e:  # 如果查询失败，捕获异常并返回错误信息
        results = f"query failed with error: {e}"
    return results  # 返回查询结果


def execute_function_call(message):
    """执行函数调用"""
    # 判断功能调用的名称是否为 "ask_database_get_sql"
    if message.tool_calls[0].function.name == "ask_database_get_sql":
        # 如果是，则获取功能调用的参数，这里是 SQL 查询
        query = json.loads(message.tool_calls[0].function.arguments)["query"]
        # 使用 ask_database 函数执行查询，并获取结果
        results = ask_database(conn, query)
    else:
        # 如果功能调用的名称不是 "ask_database"，则返回错误信息
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results  # 返回结果

In [69]:
# 创建一个空的消息列表
messages = []

# 向消息列表中添加一个系统角色的消息，内容是 "Answer user questions by generating SQL queries against the Chinook Music Database."
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})

# 向消息列表中添加一个用户角色的消息，内容是 "Hi, who are the top 5 artists by number of tracks?"
## 根据曲目数量排名前五的艺术家是谁？
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, tools)

# 从聊天响应中获取助手的消息
assistant_message = chat_response.choices[0].message
#查看大模型，调用自定义函数生成的SQL
#print(assistant_message)

# 将助手的消息添加到消息列表中
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

# 如果助手的消息中有功能调用
if assistant_message.tool_calls:
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    #print(results)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4lto09jLf2FSDjyTUYPTJU51', function=Function(arguments='{\n  "query": "SELECT artists.Name, COUNT(tracks.TrackId) as TrackCount FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.ArtistId ORDER BY TrackCount DESC LIMIT 5"\n}', name='ask_database_get_sql'), type='function')])
[('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Deep Purple', 92)]
system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

function (ask_database_get_sql): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Deep Purple', 92)]



In [53]:
database_schema_dict

[{'table_name': 'albums', 'column_names': ['AlbumId', 'Title', 'ArtistId']},
 {'table_name': 'sqlite_sequence', 'column_names': ['name', 'seq']},
 {'table_name': 'artists', 'column_names': ['ArtistId', 'Name']},
 {'table_name': 'customers',
  'column_names': ['CustomerId',
   'FirstName',
   'LastName',
   'Company',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email',
   'SupportRepId']},
 {'table_name': 'employees',
  'column_names': ['EmployeeId',
   'LastName',
   'FirstName',
   'Title',
   'ReportsTo',
   'BirthDate',
   'HireDate',
   'Address',
   'City',
   'State',
   'Country',
   'PostalCode',
   'Phone',
   'Fax',
   'Email']},
 {'table_name': 'genres', 'column_names': ['GenreId', 'Name']},
 {'table_name': 'invoices',
  'column_names': ['InvoiceId',
   'CustomerId',
   'InvoiceDate',
   'BillingAddress',
   'BillingCity',
   'BillingState',
   'BillingCountry',
   'BillingPostalCode',
   'Total']},
 {'table_name': 'invoice_i

In [136]:
# 向消息列表中添加一个用户的问题，内容是 "What is the name of the album with the most tracks?"
## 哪张专辑的曲目数量最多？
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})

# 使用 chat_completion_request 函数获取聊天响应
chat_response = chat_completion_request(messages, tools)

# 从聊天响应中获取助手的消息
#assistant_message = chat_response["choices"][0]["message"]
assistant_message = chat_response.choices[0].message

# 将助手的消息添加到消息列表中
#messages.append(assistant_message)
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

# 如果助手的消息中有功能调用(即有解析返回SQL)
if assistant_message.tool_calls:
    # 根据大模型返回的SQL，使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    #messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

我是functions参数
system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: Hi, who are the top 5 artists by number of tracks?

assistant[function_call]: {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT artists.Name, COUNT(tracks.TrackId) as NumTracks FROM artists JOIN albums ON artists.ArtistId = albums.ArtistId JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY artists.ArtistId ORDER BY NumTracks DESC LIMIT 5\"\n}"
}

function (ask_database): [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Deep Purple', 92)]

user: What is the name of the album with the most tracks?

assistant[function_call]: {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT albums.Title, COUNT(tracks.TrackId) as NumTracks FROM albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.AlbumId ORDER BY NumTracks DESC LIMIT 1\"\n}"
}

function (ask_database): [('Greatest Hits', 57)]

